In [17]:
# Imports
import pandas as pd

# Pre-Processing Airbnb Data
Source: https://insideairbnb.com/get-the-data/ (New York City Datasets, used listings & reviews data)

## Overview of Steps:
(You can click on a specific step if you'd like to skip to seeing that part)
- [Merge all data into dataset, removing duplicates by keeping most recent version of a listing](#merging-datasets)
- [Go through and remove uneeded columns](#unecessary-column-removalpruning)
- [Clean up Missing Values in dataset, impute using zero, median or average, remove row completely if needed](#handling-missing-values)
- [Converting datatypes of certain columns, like price & True/False cols, string list to list columns, datetime](#converting-datatypes-of-columns)

After these steps we can move to feature engineering, this work is done in the `FeatureEngineering.ipynb` Notebook.

## Merging Datasets

In [18]:
# Can skip to Column removal if provided full dataset
# Load Sepetember Dataset, it is the most recent
# Want to keep rows from this dataset & omit duplicates from other months
df = pd.read_csv("Datasets/AirbnbData/Sep.csv")

print(df.shape)

(37541, 75)


In [19]:
# Can skip to Column removal if provided full dataset
# Load Sepetember Dataset, it is the most recent
# Want to keep rows from this dataset & omit duplicates from other months
df = pd.read_csv("Datasets/AirbnbData/Sep.csv")

print(df.shape)

(37541, 75)


In [20]:
# Merging Step, don't run this part if only using subset
months = ["Aug", "July", "June", "May"]

for month in months:
    df_2 = pd.read_csv(f"Datasets/AirbnbData/{month}.csv")
    
    df = pd.concat([
        df, 
        df_2[df_2["id"].isin(df["id"]) == False]
    ]
    )

print(df.shape)

(42432, 75)


In [21]:
# Checking that we do not have duplicate listings
len(df["id"].unique())

42432

In [22]:
# Save df to a new dataset so we don't have to re-run merging
df.to_csv("Datasets/AirbnbData/Full_listings.csv", index=False)

## Unecessary Column Removal/Pruning 

Columns to Remove:
- listing_url
- scrape_id
- source
- host_url
- host_thumbnail_url
- host_picture_url
- calendar_updated
- calendar_last_scraped
- first_review
- last_review

These don't provide us with any useful information for the goal of creating a predictive model, these columns were selecting after looking over the [data descritpion](https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit?gid=1322284596#gid=1322284596), we will likely drop more columns later on after conducting EDA which might uncover more columns as being not impactful.

In [23]:
# Load the combined dataset 
df = pd.read_csv("Datasets/AirbnbData/Full_listings.csv")

# Get an idea of what all the columns are
df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [24]:
drop_cols = [
    "listing_url",
    "scrape_id",
    "source",
    "host_url",
    "host_thumbnail_url",
    "host_picture_url",
    "calendar_updated",
    "calendar_last_scraped",
    "first_review",
    "last_review",
    "neighbourhood"
]
df.drop(drop_cols, axis=1, inplace=True)

In [25]:
df.columns

Index(['id', 'last_scraped', 'name', 'description', 'neighborhood_overview',
       'picture_url', 'host_id', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'has_availability', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_revi

## Handling Missing Values

We will handle missing values using a variety of techniques, imputing values based on median or mean, filling values with zero or estimating a column based on the values of another column.  
Additionally, If a column has large proportion of N/A values, we may decide drop the column entirely.
The technique use will depend on the column, comments will provide a breif overview as to which technique was used and why.

In [26]:
# Check which columns have N/A values and how many/what percentage
for column in df.columns:
    missing_count = df[column].isna().sum()
    if missing_count > 0:
        print(f'{column}: {missing_count}')

name: 2
description: 1396
neighborhood_overview: 19228
picture_url: 1
host_name: 5
host_since: 5
host_location: 9503
host_about: 18465
host_response_time: 15771
host_response_rate: 15771
host_acceptance_rate: 14948
host_is_superhost: 480
host_neighbourhood: 8924
host_listings_count: 5
host_total_listings_count: 5
host_verifications: 5
host_has_profile_pic: 5
host_identity_verified: 5
bathrooms: 15539
bathrooms_text: 36
bedrooms: 6130
beds: 15703
price: 15544
minimum_minimum_nights: 1
maximum_minimum_nights: 1
minimum_maximum_nights: 1
maximum_maximum_nights: 1
minimum_nights_avg_ntm: 1
maximum_nights_avg_ntm: 1
has_availability: 5487
review_scores_rating: 13422
review_scores_accuracy: 13434
review_scores_cleanliness: 13424
review_scores_checkin: 13438
review_scores_communication: 13429
review_scores_location: 13441
review_scores_value: 13440
license: 36070
reviews_per_month: 13422


In [27]:
# Start with text columns: name, description, neighborhood_overview, host_name, picture_url, host_about
# We will change the missing values to these to just "None", when we do a sentiment/text quality analysis, we will automatically give these columns the lowest score/default score
df.fillna({
    "name": "None",
    "description": "None",
    "neighborhood_overview": "None",
    "host_name": "None",
    "host_about": "None"
}, inplace=True)

# Dropping the row without a picture url since there is only 1 row so it won't reduce our dataset size by any significant amount
# This column may or may not be used depending on whether we do feature engineering based on the photo of the airbnb
df.dropna(subset=["picture_url"], inplace=True)

In [28]:
# Next do Boolean/True False Columns: host_is_superhost, host_has_profile_pic, host_identity_verified, has_availability, license

# Columns that will will automatically be false, for these columns, we make the assumption that NaN would be false
# since having no value in these columns would correspond to not having superhost status, or not having a profile picture/license 
df.fillna({
    "host_is_superhost": "f",
    "host_has_profile_pic": "f",
    "license": "f"
}, inplace=True)

# To fill host_identity_verified, use the host_verifications which lists what methods of verification the host has, 
# if this other column is NaN or an empty list then we make the host_identity_verified column false 
df.loc[df['host_identity_verified'].isna(), 'host_identity_verified'] = df.loc[df['host_identity_verified'].isna(), 'host_verifications'].apply(
    lambda x: "f" if pd.isna(x) or x == '[]' else "t"
)

# To fill has_availability, use availability_30 to check how many days of availabilty there are in the next 30 days, if this number is > 0 then set the value to true
df.loc[df['has_availability'].isna(), 'has_availability'] = df.loc[df['has_availability'].isna(), 'availability_30'].apply(
    lambda x: "t" if x > 0 else 'f'
)

In [ ]:
# Then do host related columns that haven't been filled yet: host_since, host_location, host_response_time, host_response_rate, host_acceptance_rate, host_neighbourhood, host_listings_count, host_total_listings_count, host_verifications
# host_verifications, fill NaN with '[]' since NaN means the host hasn't been verified
df.fillna({"host_verifications": '[]'}, inplace=True)

# host_since, set value to date of corresponding last_scraped of the column, since that is when that listing was scraped
df.loc[df['host_since'].isna(), 'host_since'] = df.loc[df['host_since'].isna(), 'last_scraped']

# host_location & host_neighbourhood, want to

# host_response_time, host_response_rate, host_acceptance_rate

# host_listings_count &  host_total_listings_count

In [30]:
# Handle bed & bathroom columns: bathrooms, bathrooms_text, bedrooms, beds

In [31]:
# Handle rest of numerical columns: price, reviews_score_*, *_nights

## Converting Datatypes of columns

Some of the columns currently have type object, for example price needs to be converted to a numerical object, and the True/False columns need to be converted to Boolean/0,1 columns.

In [32]:
for column in df.columns:
    print(f'{column}: {df[column].dtype}')

id: int64
last_scraped: object
name: object
description: object
neighborhood_overview: object
picture_url: object
host_id: int64
host_name: object
host_since: object
host_location: object
host_about: object
host_response_time: object
host_response_rate: object
host_acceptance_rate: object
host_is_superhost: object
host_neighbourhood: object
host_listings_count: float64
host_total_listings_count: float64
host_verifications: object
host_has_profile_pic: object
host_identity_verified: object
neighbourhood_cleansed: object
neighbourhood_group_cleansed: object
latitude: float64
longitude: float64
property_type: object
room_type: object
accommodates: int64
bathrooms: float64
bathrooms_text: object
bedrooms: float64
beds: float64
amenities: object
price: object
minimum_nights: int64
maximum_nights: int64
minimum_minimum_nights: float64
maximum_minimum_nights: float64
minimum_maximum_nights: float64
maximum_maximum_nights: float64
minimum_nights_avg_ntm: float64
maximum_nights_avg_ntm: float64